In [ ]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

from config import *
cont_names = ['桩号', '刀盘功率', '主驱动1#电机扭矩',
                   '主驱动1#电机输出功率', '推进速度', '总推进力', '刀盘扭矩',
                   '贯入度', '推进位移']
# config = Config('full-30-s-0', debug=False, sl=30, selected_columns=True, postpond=0)

# all columns
context = Context('all_columns-full-30-s-0', Context.all_columns(), debug=False, sl=30, load_data=False)
# context = Context('all_columns-full-30-no_noise', Context.all_columns(), debug=False, sl=30, load_data=False)

In [ ]:
fns_feathers = sorted(config.fn_cycles.glob('cycle*'))[:config.num_cycles]
fmtr = DataFormatter(cycle_feathers=fns_feathers)
cycles = fmtr.cycles
# if not config.selected_columns: config.cont_names = [o for o in cycles.loc[0].columns[2:] if o not in config.dep_var]
idx = fmtr.beginning_index(postpond=config.postpond)

df_conts, stat_x = tile_with_noise(cycles, idx, config, normalize=True)
deps, stat_y = fmtr.get_y(config.dep_var, normalize=True)
deps = pd.concat([deps]*config.mulr).reset_index(drop=True)
cyc_cont = flatten_and_cat(df_conts, deps, sl=config.sl)

In [ ]:
cyc_cont.to_feather(config.fn_feather)
np.savez_compressed(config.fn_np, idx=idx, stat_x=stat_x, stat_y=stat_y)

### Without noise

In [ ]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

# all columns
# name scheme: columns-num_cycles-sl
from config import *
context = Context('all_columns-full-30-no_noise', Context.all_columns(), debug=1, sl=30, mulr=1, load_data=False)

In [ ]:
fmtr = DataFormatter(context, cycle_feathers=context.fn_cycles)
cycles = fmtr.cycles # shorten notation

idx = fmtr.beginning_index()
df_conts, stat_x = fmtr.get_x()
deps, stat_y = fmtr.get_y(normalize=True)
train_cont = flatten_and_cat(df_conts, deps, sl=context.sl)

In [ ]:
train_cont.to_feather(context.fn_feather)
np.savez_compressed(context.fn_np, idx=idx, stat_x=stat_x, stat_y=stat_y)